In [2]:
# slow down a bit when hacking something together, e.g. I forgot to add a simple function call 
# tuple unpacking is nice, but cannot be done in a nested list comprehension
# don't forget .items in  for k,v in dict.items()
# use hashlib for md5 encodings
# multiline list comprehensions don't need extra parentheses, but multiline if statements do
# np.clip min and max can be omitted by specifying None
# try except looks nice untill it obscures your real error
# parsing ints to ints instead of strings is really important
# checking whether someting is an int should be done with isinstance, not with isalpha() (fails on int)
# removing from a list while iterating can be done safely by iterating over a slice(?)
# with re make sure to use r'' literal strings
# read assignment before tinkering with networkx and discovering its not necessary
# sometimes a simple for loop works better then a list comprehension when parsing the input, and just add to concept variables
# for incrementing a string, you can use chr(ord(inp)+1)
# find repeating characters re.findall(r'([a-z])\1', password)
# regex: modify operator to nongreedy by appending ?
# ok so sometime you can bruteforce a problem....
# while manually modifying the input, make sure to change exactly the right line
# before doing bfs, check if the search space is not going to explode
# top 10 score by using a factors function!
# itertools.chain is nice for iterating over multiple iterables

In [2]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque
import itertools
import numpy as np
from matplotlib import pyplot as plt
from aoc_utils import *
import networkx as nx

In [1]:
from collections import defaultdict
from itertools import permutations
from aoc_utils import *
import re

In [14]:
mana_cost = {
    'mm' : 53,
    'drain' : 73,
    'shield' : 113,
    'poison' : 173,
    'recharge' : 229
}

In [55]:
6*5*4*500*50

3000000

In [52]:
poison * 18 + mm * 53 + drain * 2 >= 55
50 + 2 * drain - 8 *numturns
500 - MANA cost



450000

In [ ]:
Magic Missile costs 53 mana. It instantly does 4 damage.
Drain costs 73 mana. It instantly does 2 damage and heals you for 2 hit points.
Shield costs 113 mana. It starts an effect that lasts for 6 turns. While it is active, your armor is increased by 7.
Poison costs 173 mana. It starts an effect that lasts for 6 turns. At the start of each turn while it is active, it deals the boss 3 damage.
Recharge costs 229 mana. It starts an effect that lasts for 5 turns. At the start of each turn while it is active, it gives you 101 new mana.

In [174]:
class Player():
    def __init__(self, health, mana, choices = None):
        self.mana = mana
        self.damage = 0
        self.armor = 0
        self.health = health
        self.mana_used = 0
        self.cooldowns = {
            'shield' : 0,
            'poison' : 0,
            'recharge' : 0,
            'mm' : 0,
            'drain' : 0,
        }
        self.recharge_used = False
        self.shield_used = False
        self.choices = choices
    
    def turn(self):
        if self.cooldowns['recharge'] > 0: self.mana += 101
        for spell in self.cooldowns:
            if self.cooldowns[spell] > 0: self.cooldowns[spell] -= 1
        if self.cooldowns['shield'] == 0:
            self.armor = 0
        print('turn', self.cooldowns)

    def determine_spell(self,bosshealth):
        options = [spell for spell in mana_cost if mana_cost[spell] < self.mana and self.cooldowns[spell]==0]
        if self.recharge_used and 'recharge' in options:
            options.remove('recharge')
        if self.shield_used and 'shield' in options:
            options.remove('shield')
        print('options',options)
        if bosshealth <= 8:
            return 'mm'
        if self.health <= 40 and 'shield' in options:
            return 'shield'
        if self.mana <= 229+153 and 'recharge' in options:
            return 'recharge'
        if 'poison' in options: return 'poison'
        else:
            return 'mm'
            # print('cannot make choice', bosshealth, self)
            # sys.exit()

    def attack(self,bosshealth):
        spell = self.determine_spell(bosshealth)
        if self.choices:
            spell = self.choices.pop()
        print('           chosen', spell)
        if spell=='mm':
            self.mana -= 53
            self.mana_used += 53
            self.damage = 4
        elif spell=='drain':
            self.mana -= 73
            self.mana_used += 73
            self.health+=2
            self.damage = 2
        elif spell=='shield':
            self.shield_used = True
            self.mana -= 113
            self.mana_used += 113
            self.armor += 7
            self.cooldowns['shield'] = 6
            self.damage = 0
        elif spell=='poison':
            self.mana -= 173
            self.mana_used += 173
            self.cooldowns['poison'] = 6
            self.damage = 0
        elif spell=='recharge':
            self.recharge_used = True
            self.mana -= 229
            self.mana_used += 229
            self.cooldowns['recharge'] = 5
            self.damage = 0
        print('attacking for', self.damage)
    
    def __repr__(self):
        return f'\nhealth {self.health}\nmana {self.mana}\ncooldowns {self.cooldowns}'        
    def get_hit(self):
        self.health -= max(1, 8 - self.armor)
        print('boss attacks, new player health', self.health)
        if self.health <= 0:
            # this player lost
            return True
    def turn_decrease(self):
        self.health -= 1
        if self.health <= 0:
            # this player lost
            return True

    

In [184]:
choices_part1 = ['mm','mm','mm','mm','mm','poison','shield','recharge','poison']
choices_part2 = ['mm','mm','mm','drain','poison','drain','recharge','poison','shield','recharge','poison']
player = Player(50,500, choices = choices_part2)
part2 = True

def checkdeadboss(bosshealth,player):
    if bosshealth <= 0:
        print('bossdeath')
        print(player.mana_used)
        return True
        
def fight(bosshealth, player):
    bosshealth = bosshealth
    for i in range(10):
        if part2 and player.turn_decrease():
            print('dead')
            sys.exit()
        print('\nnew turn', player)
        if player.cooldowns['poison'] > 0: 
            bosshealth -= 3
            print('new boss health after poison', bosshealth)
            if checkdeadboss(bosshealth,player): sys.exit()
        player.turn()
        player.attack(bosshealth)
        bosshealth -= player.damage
        print('new boss health', bosshealth)
        if checkdeadboss(bosshealth,player): sys.exit()
        
        if player.cooldowns['poison'] > 0: 
            bosshealth -= 3
            print('new boss health after poison', bosshealth)
            if checkdeadboss(bosshealth,player): sys.exit()
        player.turn()
        if player.get_hit():
            print('dead')
            sys.exit()
        if checkdeadboss(bosshealth,player):
                sys.exit()
fight(55,player)


new turn 
health 49
mana 500
cooldowns {'shield': 0, 'poison': 0, 'recharge': 0, 'mm': 0, 'drain': 0}
turn {'shield': 0, 'poison': 0, 'recharge': 0, 'mm': 0, 'drain': 0}
options ['mm', 'drain', 'shield', 'poison', 'recharge']
           chosen poison
attacking for 0
new boss health 55
new boss health after poison 52
turn {'shield': 0, 'poison': 5, 'recharge': 0, 'mm': 0, 'drain': 0}
boss attacks, new player health 41

new turn 
health 40
mana 327
cooldowns {'shield': 0, 'poison': 5, 'recharge': 0, 'mm': 0, 'drain': 0}
new boss health after poison 49
turn {'shield': 0, 'poison': 4, 'recharge': 0, 'mm': 0, 'drain': 0}
options ['mm', 'drain', 'shield', 'recharge']
           chosen recharge
attacking for 0
new boss health 49
new boss health after poison 46
turn {'shield': 0, 'poison': 3, 'recharge': 4, 'mm': 0, 'drain': 0}
boss attacks, new player health 32

new turn 
health 31
mana 199
cooldowns {'shield': 0, 'poison': 3, 'recharge': 4, 'mm': 0, 'drain': 0}
new boss health after poison 

SystemExit: 